# BLER

In [ ]:
import re
import matplotlib.pyplot as plt

def parse_log_file(log_file):
    dl_bler_values = []
    ul_bler_values = []
    ulsch_errors = []
    with open(log_file, 'r') as f:
        for line in f:
            dl_bler_match = re.match(r'UE \w+: dlsch_rounds \d+/\d+/\d+/\d+, dlsch_errors \d+, pucch0_DTX \d+, BLER (\d+\.\d+)', line)
            if dl_bler_match:
                bler = float(dl_bler_match.group(1))
                bler_percent = bler * 100
                dl_bler_values.append(min(bler_percent, 100))

            ul_bler_match = re.match(r'UE \w+: ulsch_rounds \d+/\d+/\d+/\d+, ulsch_errors \d+, ulsch_DTX \d+, BLER (\d+\.\d+)', line)
            if ul_bler_match:
                bler = float(ul_bler_match.group(1))
                bler_percent = bler * 100
                ul_bler_values.append(min(bler_percent, 100))

            errors_match = re.search(r'ulsch_errors (\d+)', line)
            if errors_match:
                errors = int(errors_match.group(1))
                ulsch_errors.append(errors)
    return dl_bler_values, ul_bler_values, ulsch_errors


def plot_bler(dl_bler_values, ul_bler_values, filename, title=None):
    fig, ax1 = plt.subplots(figsize=(12, 6))

    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('UL and DL BLER (%)', color='black')

    ax1.plot(dl_bler_values, label='DL BLER', linestyle='-', marker='', color='tab:purple')

    ax1.plot(ul_bler_values, label='UL BLER', linestyle='-', marker='', color='tab:green')

    ax1.tick_params(axis='y', color='black')
    plt.ylim(0, 100)
    ax1.set_xlim(0,70)

    ax1.grid(True)

    plt.legend(loc='upper right')
    fig.tight_layout()

    plt.title(title if title else filename)

    plt.show()

def main(log_files):
    all_dl_bler_values = []
    all_ul_bler_values = []

    for log_file, title in log_files:
        dl_bler_values, ul_bler_values, _ = parse_log_file(log_file)
        all_dl_bler_values.append(dl_bler_values)
        all_ul_bler_values.append(ul_bler_values)
        plot_bler(dl_bler_values, ul_bler_values, log_file, title)

if __name__ == "__main__":
    log_files = [('', ' ')]
    main(log_files)


# CQI, RSRP, MCS

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np

cqi_values = []
dl_mcs_values = []
ul_mcs_values = []
rsrp_values = []


cqi_pattern = r"UE \w+: CQI (\d+)"
dl_mcs_pattern = r"UE \w+: dlsch_rounds.*MCS \(\d+\) (\d+)"
ul_mcs_pattern = r"UE \w+: ulsch_rounds.*MCS \(\d+\) (\d+)"
rsrp_pattern = r"average RSRP (-\d+)"


with open('', 'r') as file:
    for line in file:
        cqi_match = re.search(cqi_pattern, line)
        if cqi_match:
            cqi_values.append(int(cqi_match.group(1)))

        dl_mcs_match = re.search(dl_mcs_pattern, line)
        if dl_mcs_match:
            dl_mcs_values.append(int(dl_mcs_match.group(1)))

        ul_mcs_match = re.search(ul_mcs_pattern, line)
        if ul_mcs_match:
            ul_mcs_values.append(int(ul_mcs_match.group(1)))

        rsrp_match = re.search(rsrp_pattern, line)
        if rsrp_match:
            rsrp_values.append(int(rsrp_match.group(1)))


if not cqi_values and not dl_mcs_values and not ul_mcs_values and not rsrp_values:
    print("No data found to plot. Please check your log file and regular expressions.")
else:
    plt.figure(figsize=(12, 12))

    # CQI
    if cqi_values:
        plt.subplot(3, 1, 1)
        plt.plot(cqi_values, marker='o', linestyle='-', label='CQI')
        plt.xlabel('Time (s)')
        plt.ylabel('Channel Quality Indicator (CQI)')
        plt.xlim(0,)
        plt.ylim(0, max(cqi_values) + 1)
        #plt.yticks(np.arange(0, max(cqi_values) + 1, 1))  
        plt.legend()
        plt.grid(True)

    # MCS
    if dl_mcs_values or ul_mcs_values:
        plt.subplot(3, 1, 2)
        if dl_mcs_values:
            plt.plot(dl_mcs_values, marker='o', linestyle='-', label='DL MCS')
        if ul_mcs_values:
            plt.plot(ul_mcs_values, marker='s', linestyle='-', label='UL MCS')
        plt.xlabel('Time (s)')
        plt.xlim(0, )
        plt.ylabel('Modulation and Coding Scheme (MCS)')
        max_mcs = max(max(dl_mcs_values or [0]), max(ul_mcs_values or [0]))
        plt.ylim(0, max_mcs + 1)
        #plt.yticks(np.arange(0, max_mcs + 1, 1))  
        plt.legend()
        plt.grid(True)

    # RSRP
    if rsrp_values:
        plt.subplot(3, 1, 3)
        plt.plot(rsrp_values, marker='o', linestyle='-', label='RSRP', color='green')
        plt.xlabel('Time (s)')
        plt.xlim(0, )
        plt.ylabel('Reference Signal Received Power (RSRP) (dBm)')
        plt.ylim(min(rsrp_values) - 5, max(rsrp_values) + 5)
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()


# Number of PUSCH and PRACH

In [ ]:
import re
import matplotlib.pyplot as plt

def parse_log_file(file_path):
    pusch_values = {'pusch0': [], 'pusch1': [], 'pusch2': [], 'pusch3': []}
    prach_values = {'prach0': [], 'prach1': [], 'prach2': [], 'prach3': []}

    with open(file_path, 'r') as file:
        for line in file:
            match = re.search(r'\[o_du0\]\[(pusch\d)\s+(\d+)\s+prach(\d)\s+(\d+)\]', line)
            if match:
                pusch_key = match.group(1)
                pusch_value = int(match.group(2))
                prach_key = f"prach{match.group(3)}"
                prach_value = int(match.group(4))

                pusch_values[pusch_key].append(pusch_value)
                prach_values[prach_key].append(prach_value)

    return pusch_values, prach_values


def calculate_differences(values):
    differences = {}
    for key, value_list in values.items():
        differences[key] = [value_list[i] - value_list[i - 1] for i in range(1, len(value_list))]
    return differences


def plot_values(pusch_diffs, prach_diffs):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

    markers = ['s', 'o', '^', '']  

    for i, (key, values) in enumerate(pusch_diffs.items()):
        ax1.plot(values, marker=markers[i % len(markers)], label=key)
    #ax1.set_title('PUSCH Differences')
    ax1.set_xlabel('Time')
    #ax1.set_xlim(0, 80)
    ax1.set_ylabel('Num_PUSCH')
    ax1.legend()
    ax1.grid(True)

    for i, (key, values) in enumerate(prach_diffs.items()):
        ax2.plot(values, marker=markers[i % len(markers)], label=key)
    #ax2.set_title('PRACH Differences')
    ax2.set_xlabel('Time')
    #ax2.set_xlim(0, 80)
    ax2.set_ylabel('Num_PRACH')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()


def main():
    log_file_path = ''  
    pusch_values, prach_values = parse_log_file(log_file_path)

    pusch_diffs = calculate_differences(pusch_values)
    prach_diffs = calculate_differences(prach_values)

    plot_values(pusch_diffs, prach_diffs)

if __name__ == "__main__":
    main()
